<a href="https://colab.research.google.com/github/nikhil-xyz/Natural_Language_Processing/blob/main/Comment_Toxicity_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nikhil-xyz/Natural_Language_Processing.git

Cloning into 'Natural_Language_Processing'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 31 (delta 7), reused 19 (delta 4), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
df = pd.read_csv('/content/Natural_Language_Processing/data/Comment_Toxicity/train.csv')

In [4]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
df.iloc[3]['comment_text']

'"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It\'s listed in the relevant form eg Wikipedia:Good_article_nominations#Transport  "'

In [6]:
df[df.columns[2:]].iloc[3]

toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
Name: 3, dtype: int64

In [7]:
df[df['insult']==1].head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
55,0020e7119b96eeeb,Stupid peace of shit stop deleting my stuff as...,1,1,1,0,1,0
56,0020fd96ed3b8c8b,=Tony Sidaway is obviously a fistfuckee. He lo...,1,0,1,0,1,0


In [8]:
from tensorflow.keras.layers import TextVectorization

In [9]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [10]:
max_features = 200000

In [11]:
vectorizer = TextVectorization(max_tokens = max_features, output_sequence_length=1800, output_mode='int')

In [12]:
vectorizer.adapt(X.values)

In [15]:
vectorized_text = vectorizer(X.values)

In [16]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [18]:

dataset = tf.data.Dataset.from_tensor_slices((vectorized_test, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)


In [20]:
batch_x, batch_y = dataset.as_numpy_iterator().next()

In [21]:
train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.2))

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, Embedding

In [23]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(125, activation='relu'))

model.add(Dense(6, activation='sigmoid'))

In [24]:
model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
history = model.fit(train, epochs=1, validation_data=val)

6981/6981 [==============================] - 10930s 2s/step - loss: 0.0524 - accuracy: 0.9909 - val_loss: 0.0439 - val_accuracy: 0.9941


In [27]:
model.predict(test)

998/998 [==============================] - 190s 186ms/step


array([[2.0447876e-03, 3.8583439e-09, 5.6298188e-05, 3.1989305e-06,
        4.8993788e-05, 2.2349072e-06],
       [1.7508317e-03, 2.2831042e-09, 4.4386303e-05, 2.2881813e-06,
        3.8140006e-05, 1.6006940e-06],
       [3.8822781e-04, 2.7421126e-10, 8.7025483e-06, 5.9633408e-07,
        6.4779215e-06, 2.0666073e-07],
       ...,
       [3.9361388e-04, 5.6639204e-10, 1.1110075e-05, 1.0759354e-06,
        7.6990818e-06, 3.1783932e-07],
       [6.4287269e-03, 1.0245583e-07, 2.9475024e-04, 2.7688102e-05,
        2.7147410e-04, 2.0819927e-05],
       [9.9528164e-01, 9.7082250e-02, 8.9962852e-01, 1.8021200e-02,
        7.1435440e-01, 8.1832007e-02]], dtype=float32)